In [16]:
import csv
import sys
import math
import os
import librosa
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
sys.setrecursionlimit(10000)


In [17]:
import glob
import os
# reading training data
current_working_directory = os.getcwd()

base_path = current_working_directory + '/CAP6610SP24_test_set/'
prog_path = base_path + 'Progressive Rock Songs/'
non_prog_path = base_path + 'Not_Progressive_Rock/'

def fileList(path):
    matches = glob.glob(path + '/**/*', recursive=True)
    files = [file for file in matches if os.path.isfile(file) and file.endswith(('.wav', '.mp3', '.avi', '.flac', '.ogg', '.m4a', '.wma', '.mp4'))]
    return files

prog_files = fileList(prog_path)
non_prog_files = fileList(non_prog_path)

def get_prog_files():
    return prog_files

def get_non_prog_files():
    return non_prog_files

In [18]:
prog_files = get_prog_files()
non_prog_files = get_non_prog_files()

print("Number of Progressive Rock Songs: ", len(prog_files))
print("Number of Non-Progressive Rock Songs: ", len(non_prog_files))

Number of Progressive Rock Songs:  136
Number of Non-Progressive Rock Songs:  131


In [19]:
all_files = prog_files + non_prog_files
fixed_sample_rate = 44100


min_duration = 0
for i in range (len(all_files)):
    y, sr = librosa.load(all_files[i], sr=fixed_sample_rate)
    duration = librosa.get_duration(y=y, sr=sr)
    if i == 0:
        min_duration = duration
    else:
        min_duration = min(min_duration, duration)
min_duration = int(math.floor(min_duration))
print("Minimum Duration: ", min_duration)


[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
Note: Illegal Audio-MPEG-Header 0x41475061 at offset 23269468.
Note: Trying to resync...
Note: Hit end of (available) data during resync.
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
/var/folders/m0/0p062cxn2rvbyqh1m3h8cb4c0000gn/T/ipykernel_75322/4035985224.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(all_files[i], sr=fixed_sample_rate)
[src/libmpg123/id3.c:process_comment():584] er

Minimum Duration:  125


In [20]:
# feature extraction
min_duration = 10

file = open('test_features_10.csv', 'w', newline='')
header = ['filename','genre', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate']

for i in range (1, 21):
    header.append('mfcc'+str(i))

# writing giles to error.txt
error_file = open('error.txt', 'w')
error_file.close()
with file:
    writer = csv.writer(file)
    writer.writerow(header)

genre = "prog"
count = 0
# time_series_length = 30


for i in range (len(prog_files)):
    filename = prog_files[i]
    name = (filename.split('/')[-1]).replace(" ", "_")

    try: 
        y, sr = librosa.load(filename, sr=fixed_sample_rate)
        time = librosa.get_duration(y=y, sr=sr)
        chunks = []
        if time > min_duration:
            org_y = y
            iter = math.floor (time /min_duration)
            print(iter)
            current_size = time*fixed_sample_rate
            chunk_size = min_duration*fixed_sample_rate
            start = 0
            end = chunk_size
            chunk_index = 1

            while iter !=0: 
                count += 1
                chunk = y[start:end]

                chroma_stft = librosa.feature.chroma_stft(y=chunk, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=chunk, sr=sr)
                rmse = librosa.feature.rms(y=chunk)
                rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=sr)
                zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y))
                mfcc = librosa.feature.mfcc(y=chunk, sr=sr)
                to_append = f'{"prog"+name+"chunk"+str(chunk_index)} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

                # append all the 20 rows
                for e in mfcc:
                    to_append += f' {np.mean(e)}'
                
                file = open('test_features_10.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())

                
                # display spectrogram
                
                # spect = librosa.feature.melspectrogram(y=chunk, sr=sr, n_fft=2048, hop_length=512)
                # spect = librosa.power_to_db(spect, ref=np.max)
                # plt.figure(figsize=(12, 8))
                # plit.axis('off')
                # librosa.display.specshow(spect, y_axis='mel', fmax=8000, x_axis='time')
                # plt.show()
                
                chunk_index += 1

                start = end
                end = end + chunk_size
                iter -= 1
        else :
            count += 1
            y, sr = librosa.load(filename, sr=fixed_sample_rate, duration=min_duration)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rmse = librosa.feature.rms(y=y)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = 10**10*np.mean(librosa.zero_crossings(y)/len(y))
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            to_append = f'{"prog"+name} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

            # append all the 20 rows
            for e in mfcc:
                to_append += f' {np.mean(e)}'
            
            file = open('test_features_10.csv', 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())

            
            # display spectrogram
            # spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512)
            # spect = librosa.power_to_db(spect, ref=np.max)
            # plt.figure(figsize=(12, 8))
            # plt.axis('off')
            # librosa.display.specshow(spect, y_axis='mel', fmax=8000, x_axis='time')
            # plt.show()
    
    except Exception as e:
        print('error handled')
        error_file = open('error.txt', 'a')
        error_file.write(filename + '\n')
        error_file.write(str(e) + '\n')
        error_file.close()
        continue
                    



51
38
76
146
83
57
30
66
84
38
63
61
70
41
74
23
28
134
32
34
34
27
42
62


[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?


54
60
38
34
52
52
57
54
64
32
93
56
51
31
42
36
46
39
142


/Users/harlow/.local/share/virtualenvs/Harshit-ikHQR9cv/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


47
36
76
56
29
34
68
82
27
32
48
38
39
40
57
39
64
57
39
38


...

58
33
81
84
37
21
104
44
53


[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?


58


[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?


54
66
103
74


In [21]:
genre = "non_prog"
# count = 0
# time_series_length = 30


for i in range (len(non_prog_files)):
    filename = non_prog_files[i]
    name = (filename.split('/')[-1]).replace(" ", "_")

    try: 
        y, sr = librosa.load(filename, sr=fixed_sample_rate)
        time = librosa.get_duration(y=y, sr=sr)
        chunks = []
        if time > min_duration:
            org_y = y
            iter = math.floor (time /min_duration)
            print(iter)
            current_size = time*fixed_sample_rate
            chunk_size = min_duration*fixed_sample_rate
            start = 0
            end = chunk_size
            chunk_index = 1

            while iter !=0: 
                count += 1
                chunk = y[start:end]

                chroma_stft = librosa.feature.chroma_stft(y=chunk, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=chunk, sr=sr)
                rmse = librosa.feature.rms(y=chunk)
                rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=sr)
                zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y))
                mfcc = librosa.feature.mfcc(y=chunk, sr=sr)
                to_append = f'{"nonprog"+name+"chunk"+str(chunk_index)} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

                # append all the 20 rows
                for e in mfcc:
                    to_append += f' {np.mean(e)}'
                
                file = open('test_features_10.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())

                
                # display spectrogram
                
                # spect = librosa.feature.melspectrogram(y=chunk, sr=sr, n_fft=2048, hop_length=512)
                # spect = librosa.power_to_db(spect, ref=np.max)
                # plt.figure(figsize=(12, 8))
                # plit.axis('off')
                # librosa.display.specshow(spect, y_axis='mel', fmax=8000, x_axis='time')
                # plt.show()
                
                chunk_index += 1

                start = end
                end = end + chunk_size
                iter -= 1
        else :
            count += 1
            y, sr = librosa.load(filename, sr=fixed_sample_rate, duration=min_duration)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rmse = librosa.feature.rms(y=y)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = 10**10*np.mean(librosa.zero_crossings(y)/len(y))
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            to_append = f'{"nonprog"+name} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

            # append all the 20 rows
            for e in mfcc:
                to_append += f' {np.mean(e)}'
            
            file = open('test_features_10.csv', 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())

            
            # display spectrogram
            # spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512)
            # spect = librosa.power_to_db(spect, ref=np.max)
            # plt.figure(figsize=(12, 8))
            # plt.axis('off')
            # librosa.display.specshow(spect, y_axis='mel', fmax=8000, x_axis='time')
            # plt.show()
    
    except Exception as e:
        print('error handled')
        error_file = open('error.txt', 'a')
        error_file.write(filename + '\n')
        error_file.write(str(e) + '\n')
        error_file.close()
        continue
                    

153


Note: Illegal Audio-MPEG-Header 0x20202020 at offset 4065165.
Note: Trying to resync...
/var/folders/m0/0p062cxn2rvbyqh1m3h8cb4c0000gn/T/ipykernel_75322/3450540172.py:11: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(filename, sr=fixed_sample_rate)
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).


25
27


Note: Illegal Audio-MPEG-Header 0x69726420 at offset 6205317.
Note: Trying to resync...
Note: Hit end of (available) data during resync.


38
53
36
16
22
28
33


...

25
49
28
32
21
30
21
36
20
19
34
43
40
25
24
21
22
27
24
68


[src/libmpg123/id3.c:INT123_parse_new_id3():1086] error: Whoa! ID3v2 frame claims to be larger than the whole rest of the tag.


38
16
22
18
43
21
36
48


[src/libmpg123/id3.c:process_comment():584] error: No comment text / valid description?
/var/folders/m0/0p062cxn2rvbyqh1m3h8cb4c0000gn/T/ipykernel_75322/3450540172.py:11: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(filename, sr=fixed_sample_rate)
/Users/harlow/.local/share/virtualenvs/Harshit-ikHQR9cv/lib/python3.11/site-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


26
60
24
159
18
20
26
198
22
